In [2]:
import matplotlib.pyplot as plt


In [3]:
speedup = [3.5, 2.8, 2.2, 1.85, 1.75, 1.65, 1.6, 1.4, 1.35, 1.20, 1.09, 1.0]
plt.plot([i+1 for i in range(len(speedup))], speedup)
plt.grid(b=True)

In [4]:
corr = [0.22, 0.52, 0.53, 0.54, 0.55, 0.7, 0.81, 0.89, 0.9, 0.93, 1.0, 1.0]
corr30 = [0.35, 0.65, 0.71, 0.73, 0.74, 0.78, 0.82, 0.89, 0.9, 0.91, 1.0, 1.0]
plt.plot([i+1 for i in range(len(corr))], corr)
plt.plot([i+1 for i in range(len(corr))], corr30)
plt.grid(b=True)

In [5]:
fig, ax1 = plt.subplots()
ax1.plot([i+1 for i in range(len(corr))], corr)
ax1.plot([i+1 for i in range(len(corr))], corr30)
ax1.set_ylabel('Correlation')
ax1.set_xlabel('Subnetwork size ($\lambda$)')
ax1.legend(['Correlation', 'Correlation of Top 30'], bbox_to_anchor=(0,1.02, 1, 0.2), loc="lower left")

ax2 = ax1.twinx()
ax2.plot([i+1 for i in range(len(speedup))], speedup, 'g-')
ax2.set_ylabel('Speedup')
ax2.legend(['Run time speedup'], bbox_to_anchor=(0.6,1.02, 1, 0.2), loc="lower left")

plt.grid(b=True)
plt.savefig('corr_runtime.png')

In [30]:
from xml.etree import ElementTree as ET



In [65]:
from collections import defaultdict
import numpy as np

def process_route(element, depart, veh_id):
    if element.tag == 'route' and 'exitTimes' in element.keys():
        depart = float(depart)
        edges = [edge.split('_')[0] for edge in element.get('edges').split(' ')]
        exit_times = np.array([depart] + [float(t) for t in element.get('exitTimes').split(' ')])
        travel_times = np.diff(exit_times)
        if len(edges) != len(travel_times):
            #print('Somethings wrong!')
            print(veh_id)
        return dict(zip(edges, travel_times))
#tree = ET.parse('../output/net_dump/vehroutes100.xml')
#traveltime_1_1_0_10_9_False.json
tree = ET.parse('../output/net_dump/lmbd9/traveltime_1_1_0_10_9_False.json.xml')
root = tree.getroot()

nominal_tt = defaultdict(dict)
counts = defaultdict(int)
for veh in root:
    depart = veh.get('depart')
    counts[veh.tag] += 1
    for el in list(veh):
        counts[el.tag] += 1
        if el.tag == 'routeDistribution':
            for e in list(el):
                nominal_tt[veh.get('id')] =  process_route(e, depart, veh.get('id'))
        else:
            nominal_tt[veh.get('id')] = process_route(el, depart, veh.get('id'))
        


27642
51490
53432
53156
46440
31867
54722
52714
43357
44996
53587
50004
52600
53724
42974
53906
46110
46282
46310
44707
32407
55102
53562
49870
49674
51540
51264
47054
51912
46374
47534
47796
50742
53934
52500
48114
52760
34004
54579
43574
51857
54854
50942
54210
47896
47292
46594
50067
46949
35206
50180
55116
54697
52612
29547
35072
49206
34427
53392
47154
46490
54542
54887
50014
52630
52322
50114
53526
49017
50117
48794
32449
54619
54069
54197
46982
52259
53959
54016
27399
53487
53600
46599
51664
49752
54060
50650
54710
48507
53884
34227
19124
50227
19224
49004
50269
47750
49979
49476
46116
48446
49228
46202
54230
34142
49760
49287
50512
48807
51472
49371
46507
51887
47550
51504
47768
50594
51760
46477
47517
47809
45687
39216
34969
34767
46449
50216
52764
48339
51044
53857
39112
52866
53019
47664
47099
55017
50324
47450
50620
34240
50064
53417
49542
45757
51676
49560
49282
37087
41276
25694
50801
49222
52680
21207
52129
46240
48513
43029
47730
35119
46694
23266
46340
50100
48689
5005

In [37]:
import json 
json.dump(nominal_tt, open('../output/net_dump/vehroutes_new.json', 'w'))

In [23]:
list(root[0])[0].get('exitTimes').split(' ')

['23111.00', '23224.00', '23274.00']

In [40]:
new = json.load(open('../output/net_dump/lmbd2/traveltime_1_1_28800_57600_2_False.json'))
old = json.load(open('../output/net_dump/lmbd2/traveltime_1_1_28800_57600_2_False_old.json'))
change = 0
total = 0
for veh in new:
    total += 1
    if old[veh] != new[veh]:
        change += 1


In [52]:
from pathlib import Path
sim_times = {}

base_path = Path('../output/net_dump/lmbd1')
for fn in base_path.glob('*.json'):
    data = json.load(fn.open())
    sim_times[str(fn)] = data['sim_time']



In [53]:
import operator
max(sim_times.items(), key=operator.itemgetter(1))[0]

'../output/net_dump/lmbd1/traveltime_55_1_57600_86400_1_False.json'

In [55]:
sim_times['../output/net_dump/lmbd1/traveltime_55_1_57600_86400_1_False.json']

457.0282552242279

In [37]:
import xml.etree.ElementTree as ET
from pathlib import Path
edge_name, start, end = '59_1', 57600, 86400
lmbd = 1
filename = Path(f'../output/net_dump/high_correlation/lmbd{lmbd}/traveltime_{edge_name}_{start}_{end}_{lmbd}_False.json.xml')
tree = ET.parse(filename)
root = tree.getroot()

In [38]:
total_vehicles = []
through_5_1 = []
for vehicle in root.iter('vehicle'):
    if float(vehicle.attrib['depart']) > start:
        total_vehicles.append(vehicle.attrib['id'])
        for route in vehicle:
            edges = set(route.get('edges', default='').split(' '))
            if edge_name in edges:
                through_5_1.append(vehicle.attrib['id'])
            

In [39]:
through_5_1[9]

'92034'

In [8]:
travel_times = {}

In [32]:
#edge_name, start, end = '5_1', 57600, 86400
edge_name, start, end = '59_1', 0, 10
lmbd = 1
filename = Path(f'../output/net_dump/high_correlation/lmbd{lmbd}/traveltime_{edge_name}_{start}_{end}_{lmbd}_False.json.xml')
tree = ET.parse(filename)
root = tree.getroot()
for vehicle in root.iter('vehicle'):
    #if vehicle.get('id') == '91950':
    if vehicle.get('id') == '94766':
        travel_times[(lmbd, start, end)] = float(vehicle.get('arrival')) - float(vehicle.get('depart'))


In [33]:
travel_times

{(1, 57600, 86400): 105.0,
 (3, 57600, 86400): 2343.0,
 (9, 57600, 86400): 2349.0,
 (1, 0, 10): 104.0,
 (3, 0, 10): 562.0,
 (9, 0, 10): 570.0}